In [10]:
from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(42)
from sklearn.neural_network import MLPRegressor
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.optimizers import rmsprop
from keras import backend as ker
from math import sqrt
import numpy as np
import tensorflow as tf
import pandas as pd
import gc
from sklearn.model_selection import train_test_split
RANDOM_SEED= 40
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.optimizers import rmsprop
from keras import backend as ker

def remov_nan (dataset):
    '''
    to remove all NaN Values in a 
    Time Serie Dataframe
    '''
    n = dataset.isnull().sum() 
    data = dataset[0:(len(dataset)-n)]
    return data


def detrend(insample_data):
    """
    Calculates a & b parameters of LRL

    :param insample_data:
    :return:
    """
    x = np.arange(len(insample_data))
    a, b = np.polyfit(x, insample_data, 1)
    return a, b


def deseasonalize(original_ts, ppy):
    """
    Calculates and returns seasonal indices

    :param original_ts: original data
    :param ppy: periods per year
    :return:
    """
    """
    # === get in-sample data
    original_ts = original_ts[:-out_of_sample]
    """
    if seasonality_test(original_ts, ppy):
        # print("seasonal")
        # ==== get moving averages
        ma_ts = moving_averages(original_ts, ppy)

        # ==== get seasonality indices
        le_ts = original_ts * 100 / ma_ts
        le_ts = np.hstack((le_ts, np.full((ppy - (len(le_ts) % ppy)), np.nan)))
        le_ts = np.reshape(le_ts, (-1, ppy))
        si = np.nanmean(le_ts, 0)
        norm = np.sum(si) / (ppy * 100)
        si = si / norm
    else:
        # print("NOT seasonal")
        si = np.full(ppy, 100)

    return si


def moving_averages(ts_init, window):
    """
    Calculates the moving averages for a given TS

    :param ts_init: the original time series
    :param window: window length
    :return: moving averages ts
    """
    if len(ts_init) % 2 == 0:
        ts_ma = pd.rolling_mean(ts_init, window, center=True)
        ts_ma = pd.rolling_mean(ts_ma, 2, center=True)
        ts_ma = np.roll(ts_ma, -1)
    else:
        ts_ma = pd.rolling_mean(ts_init, window, center=True)

    return ts_ma


def seasonality_test(original_ts, ppy):
    """
    Seasonality test

    :param original_ts: time series
    :param ppy: periods per year
    :return: boolean value: whether the TS is seasonal
    """
    s = acf(original_ts, 1)
    for i in range(2, ppy):
        s = s + (acf(original_ts, i) ** 2)

    limit = 1.645 * (sqrt((1 + 2 * s) / len(original_ts)))

    return (abs(acf(original_ts, ppy))) > limit


def acf(data, k):
    """
    Autocorrelation function

    :param data: time series
    :param k: lag
    :return:
    """
    m = np.mean(data)
    s1 = 0
    for i in range(k, len(data)):
        s1 = s1 + ((data[i] - m) * (data[i - k] - m))

    s2 = 0
    for i in range(0, len(data)):
        s2 = s2 + ((data[i] - m) ** 2)

    return float(s1 / s2)

def windows_for_forecasts(data, in_num, fh):
    """
    Splits the series into train and test sets. Each step takes multiple points as inputs
    """
    x_train, y_train = data[:-1], np.roll(data, -in_num)[:-in_num]


    # reshape input to be [samples, time steps, features] (N-NF samples, 1 time step, 1 feature)
    x_train = np.reshape(x_train, (-1, 1))
  
    temp_train = np.roll(x_train, -1)
    for x in range(1, in_num):
        x_train = np.concatenate((x_train[:-1], temp_train[:-1]), 1)
        temp_train = np.roll(temp_train, -1)[:-1]

    return x_train, y_train



def split_into_train_test(data, in_num, fh):
    """
    Splits the series into train and test sets. Each step takes multiple points as inputs

    :param data: an individual TS
    :param fh: number of out of sample points
    :param in_num: number of input points for the forecast
    :return:
    """
    train, test = data[:-fh], data[-(fh + in_num):]
    x_train, y_train = train[:-1], np.roll(train, -in_num)[:-in_num]
    x_test, y_test = train[-in_num:], np.roll(test, -in_num)[:-in_num]

    # reshape input to be [samples, time steps, features] (N-NF samples, 1 time step, 1 feature)
    x_train = np.reshape(x_train, (-1, 1))
    x_test = np.reshape(x_test, (-1, 1))
    temp_test = np.roll(x_test, -1)
    temp_train = np.roll(x_train, -1)
    for x in range(1, in_num):
        x_train = np.concatenate((x_train[:-1], temp_train[:-1]), 1)
        x_test = np.concatenate((x_test[:-1], temp_test[:-1]), 1)
        temp_test = np.roll(temp_test, -1)[:-1]
        temp_train = np.roll(temp_train, -1)[:-1]

    return x_train, y_train, x_test, y_test


def rnn_bench(x_train, y_train, x_test, fh, input_size):
    """
    Forecasts using 6 SimpleRNN nodes in the hidden layer and a Dense output layer

    :param x_train: train data
    :param y_train: target values for training
    :param x_test: test data
    :param fh: forecasting horizon
    :param input_size: number of points used as input
    :return:
    """
    # reshape to match expected input
    x_train = np.reshape(x_train, (-1, input_size, 1))
    x_test = np.reshape(x_test, (-1, input_size, 1))

    # create the model
    model = Sequential([
        SimpleRNN(6, input_shape=(input_size, 1), activation='linear',
                  use_bias=False, kernel_initializer='glorot_uniform',
                  recurrent_initializer='orthogonal', bias_initializer='zeros',
                  dropout=0.0, recurrent_dropout=0.0),
        Dense(1, use_bias=True, activation='linear')
    ])
    opt = rmsprop(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)

    # fit the model to the training data
    model.fit(x_train, y_train, epochs=1000, batch_size=100)
   # model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=0)

    # make predictions
    y_hat_test = []
    last_prediction = model.predict(x_test)[0]
    for i in range(0, fh):
        y_hat_test.append(last_prediction)
        x_test[0] = np.roll(x_test[0], -1)
        x_test[0, (len(x_test[0]) - 1)] = last_prediction
        last_prediction = model.predict(x_test)[0]

    return np.asarray(y_hat_test)


def mlp_bench(x_train, y_train, x_test, fh):
    """
    Forecasts using a simple MLP which 6 nodes in the hidden layer

    :param x_train: train input data
    :param y_train: target values for training
    :param x_test: test data
    :param fh: forecasting horizon
    :return:
    """
    y_hat_test = []

    model = MLPRegressor(hidden_layer_sizes=6, activation='identity', solver='adam',
                         max_iter=100, learning_rate='adaptive', learning_rate_init=0.001,
                         random_state=42)
    model.fit(x_train, y_train)

    last_prediction = model.predict(x_test)[0]
    for i in range(0, fh):
        y_hat_test.append(last_prediction)
        x_test[0] = np.roll(x_test[0], -1)
        x_test[0, (len(x_test[0]) - 1)] = last_prediction
        last_prediction = model.predict(x_test)[0]

    return np.asarray(y_hat_test)


   



def smape(a, b):
    """
    Calculates sMAPE

    :param a: actual values
    :param b: predicted values
    :return: sMAPE
    """
    a = np.reshape(a, (-1,))
    b = np.reshape(b, (-1,))
    return np.mean(2.0 * np.abs(a - b) / (np.abs(a) + np.abs(b))).item()


def mase(insample, y_test, y_hat_test, freq):
    """
    Calculates MAsE

    :param insample: insample data
    :param y_test: out of sample target values
    :param y_hat_test: predicted values
    :param freq: data frequency
    :return:
    """
    y_hat_naive = []
    for i in range(freq, len(insample)):
        y_hat_naive.append(insample[(i - freq)])

    masep = np.mean(abs(insample[freq:] - y_hat_naive))

    return np.mean(abs(y_test - y_hat_test)) / masep




In [69]:
import time
def main(data_path,fh,freq,df,series_type,forecasts_path):
    print('Prepare Data')
    data_all=pd.read_csv(data_path, skiprows=0, index_col =0)
    in_size = 5    # number of points used as input for each forecast

    err_RNN_sMAPE = []
    err_RNN_MASE = []
    

    
    counter = 0
    # ===== Main loop which goes through all timeseries =====
    for j in range(len(data_all)):
        start = time.time()
        
        
        ts = data_all.iloc[j, :]
        ts = remov_nan(ts)

        # remove seasonality
        seasonality_in = deseasonalize(ts, freq)

        for i in range(0, len(ts)):
            ts[i] = ts[i] * 100 / seasonality_in[i % freq]

        # detrending
        a, b = detrend(ts)

        for i in range(0, len(ts)):
            ts[i] = ts[i] - ((a * i) + b)

        x_train, y_train = windows_for_forecasts(ts, in_size, fh)
        
        
       
        # RNN benchmark - Produce forecasts
        print('Train LSTM')
        #y_hat_test_RNN = np.reshape(rnn_bench(x_train, y_train, x_test, fh, in_size), (-1))
        windowForForecast=ts[len(ts)-in_size:]
        print('windowForForecast',windowForForecast.shape)
        y_hat_test_RNN=LSTM_NN(x_train, y_train, windowForForecast, fh, in_size)
        y_hat_test_RNN = np.reshape(y_hat_test_RNN, (-1))

        print('series_type',series_type)
        df.iloc[j,0]=str(series_type[0])+str(j)
        print('y_hat_test_RNN',y_hat_test_RNN)
        print('y_hat_test_RNN shape',y_hat_test_RNN.shape)
        print('df shape',df.iloc[j].shape)
        df.iloc[j,1:(len(y_hat_test_RNN)+1)]=y_hat_test_RNN
        df.to_csv(forecasts_path, index=False)
        print('Test Models')
     

        # add trend
        for i in range(0, len(ts)):
            ts[i] = ts[i] + ((a * i) + b)

        for i in range(0, fh):
            #y_hat_test_MLP[i] = y_hat_test_MLP[i] + ((a * (len(ts) + i + 1)) + b)
            y_hat_test_RNN[i] = y_hat_test_RNN[i] + ((a * (len(ts) + i + 1)) + b)

        # add seasonality
        for i in range(0, len(ts)):
            ts[i] = ts[i] * seasonality_in[i % freq] / 100

        for i in range(len(ts), len(ts) + fh):
            y_hat_test_RNN[i - len(ts)] = y_hat_test_RNN[i - len(ts)] * seasonality_in[i % freq] / 100

        # check if negative or extreme
        for i in range(len(y_hat_test_RNN)):
       
            if y_hat_test_RNN[i] < 0:
                y_hat_test_RNN[i] = 0
                
            
            if y_hat_test_RNN[i] > (1000 * max(ts)):
                y_hat_test_RNN[i] = max(ts)

    return y_hat_test_RNN


In [70]:
from keras.callbacks import EarlyStopping

def LSTM_NN(x_train, y_train, x_test, fh, in_back):

    
    
    # reshape to match expected input
    x_train = np.reshape(x_train, (-1, in_back, 1))
    x_test = np.reshape(x_test, (-1, in_back, 1))
      
     # create the model and parametrize it
    model = Sequential()
    
    #model.add(LSTM(input_shape=(in_back,1),output_dim=100,return_sequences=True))
    #model.add(LSTM(input_shape=(in_back,1),output_dim=50,return_sequences=True))

    model.add(LSTM(input_shape=(in_back,1),units=512))
    #model.add(LSTM(512))


    model.add(Dropout(0.5))
    model.add(Activation('linear'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    # fit the model to the training data
    early_stopping_monitor = EarlyStopping(monitor='loss', mode='min',patience=4)

    model.fit(x_train, y_train, epochs=1500, batch_size=200,callbacks=[early_stopping_monitor],verbose=0)
    #model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=0)
    

    # make predictions
    y_hat_test = []
    last_prediction = model.predict(x_test)[0]
    for i in range(0, fh):
        y_hat_test.append(last_prediction)
        x_test[0] = np.roll(x_test[0], -1)
        x_test[0, (len(x_test[0]) - 1)] = last_prediction
        last_prediction = model.predict(x_test)[0]
    
    forecast = []
    


    return np.asarray(y_hat_test)





In [71]:
p=['Daily','Weekly','Hourly','Monthly','Quaterly','Yearly']
fcs=[14,13,48,18,8,1]
frqs=[1,1,24,12,4,1]
size=20
all_forecasts=np.zeros((100000,49))
all_forecasts.fill(np.nan)
all_forecasts
columns=['id']
for i in range(1,49):
    columns.append('F'+str(i))
columns
df=pd.DataFrame(all_forecasts,columns=columns)
df.fillna
forecasts_path='../results/forecasts-'+str(size)+'.csv'
#df.to_csv(forecasts_path, index=False)

test1=[]
test2=[]
for x in range(0,len(p)):
    print(p[x])
    path="".join(["../data/csv_",str(size),"_sample/", str(p[x]),'-train-',str(size),'-sample.csv'])
    
    main(path,fcs[x],frqs[x],df,p[x],forecasts_path)
    
    



Daily
Prepare Data


/home/jupyter/anaconda3/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:92: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=1,center=True).mean()
/home/jupyter/anaconda3/envs/jupyter/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


Train LSTM
windowForForecast (5,)
series_type Daily
y_hat_test_RNN [939.0928  937.1328  935.7594  934.28033 932.8767  931.50525 930.22485
 929.039   927.9828  927.04895 926.22815 925.54755 924.9829  924.5196 ]
y_hat_test_RNN shape (14,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Daily
y_hat_test_RNN [411.51938 385.9243  432.64078 491.6146  513.24005 472.46765 454.9778
 445.74927 433.24213 424.82272 427.32288 454.21292 480.77533 481.98785]
y_hat_test_RNN shape (14,)
df shape (49,)
Test Models


/home/jupyter/anaconda3/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:88: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=1,center=True).mean()
/home/jupyter/anaconda3/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:89: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=2,center=True).mean()


Train LSTM
windowForForecast (5,)
series_type Daily
y_hat_test_RNN [714.983   689.9539  684.31726 681.1625  678.4194  677.0754  676.32654
 675.83154 675.5052  675.29675 675.16437 675.08057 675.0278  674.99445]
y_hat_test_RNN shape (14,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Daily
y_hat_test_RNN [491.48785 485.8333  483.0422  481.74847 481.0738  480.69873 480.48038
 480.35342 480.27835 480.2326  480.2048  480.18774 480.1774  480.171  ]
y_hat_test_RNN shape (14,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Daily
y_hat_test_RNN [-291.63025 -294.69022 -296.31787 -297.20218 -297.7425  -298.15338
 -298.5008  -298.8117  -299.093   -299.34772 -299.58145 -299.80005
 -300.00464 -300.19653]
y_hat_test_RNN shape (14,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Daily
y_hat_test_RNN [-111.07876  -105.14586   -99.94797   -94.17185   -87.62311   -80.885155
  -75.047325  -69.19708   -62.818283  -55.698326  -46

...

series_type Monthly
y_hat_test_RNN [-167.33057  -162.71555  -155.55203  -143.54248  -124.09134   -96.95714
  -76.755646  -83.38384  -103.95899  -140.39642  -159.54962  -164.9352
 -164.13127  -156.57329  -146.5036   -130.39026  -107.19244   -84.99782 ]
y_hat_test_RNN shape (18,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Monthly
y_hat_test_RNN [59.84594  57.842693 57.328453 56.60876  55.735016 55.949615 55.853107
 55.774773 55.776165 55.785538 55.771076 55.771378 55.7729   55.771812
 55.77131  55.771675 55.771576 55.771492]
y_hat_test_RNN shape (18,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Monthly
y_hat_test_RNN [98.21087  97.05809  95.58485  93.87512  93.26293  92.98489  92.64319
 92.485916 92.40661  92.3328   92.29043  92.26773  92.24991  92.23869
 92.23225  92.22766  92.22467  92.222855]
y_hat_test_RNN shape (18,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Monthly
y_hat_test_RNN [-234.48563 -

/home/jupyter/anaconda3/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:92: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=4,center=True).mean()


series_type Quaterly
y_hat_test_RNN [-48.785614  -19.146646   -8.3629265   5.5011053  21.35555    28.103687
  27.977356   25.137577 ]
y_hat_test_RNN shape (8,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Quaterly
y_hat_test_RNN [ 21.05193   51.30295   29.3692   -10.278424  23.068336  17.35085
  39.08306   50.48009 ]
y_hat_test_RNN shape (8,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Quaterly
y_hat_test_RNN [-35.979126 -37.0401   -38.68551  -40.327637 -40.047764 -40.27954
 -40.29599  -40.359592]
y_hat_test_RNN shape (8,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Quaterly
y_hat_test_RNN [ 63.343075   25.792728  -11.2961645 -72.99383   -93.53572   -88.93789
  -8.875158   29.17389  ]
y_hat_test_RNN shape (8,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Quaterly
y_hat_test_RNN [-53.551285 -49.927235 -45.729893 -44.19748  -45.461544 -41.441246
 -41.316418 -40.51721 ]
y_hat_

/home/jupyter/anaconda3/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:88: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=4,center=True).mean()


series_type Quaterly
y_hat_test_RNN [-62.01335   49.07371   19.4212   -92.26602   19.606253  62.853226
 -71.60842  -89.88366 ]
y_hat_test_RNN shape (8,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Quaterly
y_hat_test_RNN [68.23239  68.38317  68.360214 68.32397  68.230484 14.820483  4.433389
  5.853091]
y_hat_test_RNN shape (8,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Quaterly
y_hat_test_RNN [-62.405666 -54.128277 -54.849224 -53.795666 -45.15439  -30.360214
 -20.228403 -12.976864]
y_hat_test_RNN shape (8,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Quaterly
y_hat_test_RNN [269.064    249.3041   211.26297  177.7451   137.21263  105.754486
  81.766106  55.351875]
y_hat_test_RNN shape (8,)
df shape (49,)
Test Models
Train LSTM
windowForForecast (5,)
series_type Quaterly
y_hat_test_RNN [-33.9016   -32.127026 -30.957378 -30.484133 -30.262325 -29.956015
 -29.82815  -29.754524]
y_hat_test_RNN shape (8,)

In [ ]:
print('smape',test1,'mase',test2)

In [62]:
df=[[927.9543 ],
 [919.9753 ],
 [916.0806 ],
 [914.976  ],
 [915.10547],
 [915.569  ],
 [915.94727],
 [916.1538 ],
 [916.2281 ],
 [916.23413],
 [916.2175 ],
 [916.20074],
 [916.1903 ],
 [916.1859 ]]
df.flatten()

AttributeError: 'list' object has no attribute 'flatten'

In [55]:
df=pd.DataFrame([[0,0,0],[0,0,0]],columns=['A','B','C'])
a=[1,2]
df.iloc[0,1]='dadad'[0]+str(1)
df.iloc[0,1:len(a)+1]=a
df

,A,B,C
0,0,1,2
1,0,0,0


In [ ]:

v1 = array(1, dim=c(8,length(p)*size))
v2 = array(1, dim=c(8,length(p)*size))
    glimpse(v1)

for (x in 1:length(p)){   
    v1[,((x-1)*size+1):(x*size)]=test1[[x]]
    v2[,((x-1)*size+1):(x*size)]=test2[[x]]
}
glimpse(v1)
glimpse(v2)

Total_mase=v2
Total_smape=v1
result=array(NA,dim = c(length(Names_benchmarks), 4))
print("########### sMAPE ###############")
for (i in range(1,len(Names_benchmarks)){
  smape=round(np.mean(Total_smape[i,]), 3)
  mase=round(np.mean(Total_mase[i,]), 3)
  owa=round(((np.mean(Total_mase[i,])/mean(Total_mase[3,]))+(mean(Total_smape[i,])/mean(Total_smape[3,])))/2, 3)
  
  result[i,1]=Names_benchmarks[i]
  result[i,2]=smape
  result[i,3]=mase
  result[i,4]=owa  
  
  print(paste(Names_benchmarks[i], smape, mase, owa))
}
write.csv(result, file = paste("../results/R-Benchmark-",size,'-sample.csv'))

In [ ]:
def main_all():
    
    print("### Load of Dataset  ###")
    #df_yearly = pd.read_csv("../M4-methods/csv_20_sample/Yearly-train-20-sample.csv", skiprows=0, index_col =0)
    #df_quaterly = pd.read_csv("../M4-methods/csv_20_sample/Quaterly-train-20-sample.csv", skiprows=0, index_col =0)
    df_monthly = pd.read_csv("../M4-methods/csv_20_sample/Monthly-train-20-sample.csv", skiprows=0, index_col =0)
    df_weekly = pd.read_csv("../M4-methods/csv_20_sample/Weekly-train-20-sample.csv", skiprows=0, index_col =0)
    df_daily = pd.read_csv("../M4-methods/csv_20_sample/Daily-train-20-sample.csv", skiprows=0, index_col =0)
    df_hourly = pd.read_csv("../M4-methods/csv_20_sample/Hourly-train-20-sample.csv", skiprows=0, index_col =0)
 
    
    D=[]
   # D.append(df_yearly)
   # D.append(df_quaterly)
    D.append(df_monthly)
    D.append(df_weekly)
    D.append(df_daily)
    D.append(df_hourly)
    
    columnsname= ["Data_Type","sMapeLSTM","Mase LSTM"]
    ds = pd.DataFrame(columns=columnsname )
    ds.to_csv('outpoutM4.csv')
    print(ds.shape)
    
    for i in range (len(D)):
        
        if i==0:
            print( "*** Beginn of Monthly dataset ***")
            a,b = main(df_monthly,18,12,i)
            p= ["Monthly_data",a,b]
            ds.iloc[:,i]= p
            ds=ds.round(4)
            ds.to_csv('outputM4LSTM.csv', mode='a', header=False)
        if i==1:
            print( "*** Beginn of Weekly dataset ***")
            a,b = main(df_weekly,13,1,i)
            p= ["Weekly_data",a,b]
            ds.iloc[:,i]= p
            ds=ds.round(4)
            ds.to_csv('outputM4LSTM.csv', mode='a', header=False)
        if i==2:
            print( "*** Beginn of Daily dataset ***")
            a,b= main(df_daily,14,1,i)
            p= ["Daily_data",a,b]
            ds.iloc[:,i]= p
            ds=ds.round(4)
            ds.to_csv('outputM4LSTM.csv', mode='a', header=False)
        if i==3:
            print( "*** Beginn of Hourly dataset ***")
            a,b = main(df_hourly,48,24,i)
            p= ["Hourly_data",a,b]
            ds.iloc[:,i]= p
            ds=ds.round(4)
            ds.to_csv('outputM4LSTM.csv', mode='a', header=False)
    print("Done")

In [ ]:
all_forecasts=np.zeros((100000,49))
all_forecasts.fill(np.nan)
all_forecasts
columns=['id']
for i in range(1,49):
    columns.append('F'+str(i))
columns
df=pd.DataFrame(all_forecasts,columns=columns)
df.fillna

In [ ]:
main_all()

In [ ]:
main_all()

In [ ]:
if i==0:
            print( "*** Beginn of yearly dataset ***")
            a,b = main(df_yearly,6,1,i)
            p= ["Yearly_data",a,b]
            ds.iloc[:,i]= p
            ds=ds.round(4)
            ds.to_csv('outputM4LSTM.csv', mode='a', header=False)
        if i==1:
            print( "*** Beginn of Quarterly dataset ***")
            a,b = main(df_quaterly,8,4,i)
            p= ["Quarterly_data",a,b]
            ds.iloc[:,i]= p
            ds=ds.round(4)
            ds.to_csv('outputM4LSTM.csv', mode='a', header=False)

In [ ]:
import time
def main(data_path,fh,freq,df,series_type,forecasts_path):
    print('Prepare Data')
    data_all=pd.read_csv(data_path, skiprows=0, index_col =0)
    in_size = 5    # number of points used as input for each forecast

    err_RNN_sMAPE = []
    err_RNN_MASE = []
    

    
    counter = 0
    # ===== Main loop which goes through all timeseries =====
    for j in range(len(data_all)):
        start = time.time()
        
        
        ts = data_all.iloc[j, :]
        ts = remov_nan(ts)

        # remove seasonality
        seasonality_in = deseasonalize(ts, freq)

        for i in range(0, len(ts)):
            ts[i] = ts[i] * 100 / seasonality_in[i % freq]

        # detrending
        a, b = detrend(ts)

        for i in range(0, len(ts)):
            ts[i] = ts[i] - ((a * i) + b)

        x_train, y_train = windows_for_forecasts(ts, in_size, fh)
        
        
       
        # RNN benchmark - Produce forecasts
        print('Train LSTM')
        #y_hat_test_RNN = np.reshape(rnn_bench(x_train, y_train, x_test, fh, in_size), (-1))
        windowForForecast=ts[len(ts)-in_size:]
        print(windowForForecast,windowForForecast.shape)
        lstm=LSTM_NN(x_train, y_train, windowForForecast, fh, in_size)
        df.loc[j,0]=series_type[0]+str(j)
        df.loc[j,1:len(forecast)+1]=forecast
        df.to_csv(forecasts_path,mode='a', index=False)
        y_hat_test_RNN = np.reshape(lstm, (-1))
        print('Test Models')
     

        # add trend
        for i in range(0, len(ts)):
            ts[i] = ts[i] + ((a * i) + b)

        for i in range(0, fh):
            #y_hat_test_MLP[i] = y_hat_test_MLP[i] + ((a * (len(ts) + i + 1)) + b)
            y_hat_test_RNN[i] = y_hat_test_RNN[i] + ((a * (len(ts) + i + 1)) + b)

        # add seasonality
        for i in range(0, len(ts)):
            ts[i] = ts[i] * seasonality_in[i % freq] / 100

        for i in range(len(ts), len(ts) + fh):
            y_hat_test_RNN[i - len(ts)] = y_hat_test_RNN[i - len(ts)] * seasonality_in[i % freq] / 100

        # check if negative or extreme
        for i in range(len(y_hat_test_RNN)):
       
            if y_hat_test_RNN[i] < 0:
                y_hat_test_RNN[i] = 0
                
            
            if y_hat_test_RNN[i] > (1000 * max(ts)):
                y_hat_test_RNN[i] = max(ts)

        x_train, y_train, x_test, y_test = split_into_train_test(ts, in_size, fh)

        # Calculate errors
       # err_MLP_sMAPE.append(smape(y_test, y_hat_test_MLP))
        err_RNN_sMAPE.append(smape(y_test, y_hat_test_RNN))
      #  err_MLP_MASE.append(mase(ts[:-fh], y_test, y_hat_test_MLP, freq))
        err_RNN_MASE.append(mase(ts[:-fh], y_test, y_hat_test_RNN, freq))

        # memory handling
        ker.clear_session()
        tf.reset_default_graph()
        gc.collect()

        counter = counter + 1
        #**********************************************************************
        print("-------------TS ID: ", counter, "-------------")
        print(" sMAPE_RNN:",err_RNN_sMAPE[-1]," MASE_RNN:",err_RNN_MASE[-1])
        print("Time:",time.time()-start)
        #break
        
        #********************************************************************    
    print("\n\n---------FINAL RESULTS---------")
    print("=============sMAPE=============\n")
    print("#### RNN ####\n", np.mean(err_RNN_sMAPE), "\n")
    print("==============MASE=============")
    print("#### RNN ####\n", np.mean(err_RNN_MASE), "\n")
    return np.mean(err_RNN_sMAPE),np.mean(err_RNN_MASE)